In [1]:
print("hi")

import os
import pandas as pd
import subprocess
import time
import re
import numpy as np

import csv

from scripts import combat_info
from scripts import combat_quick_apply
from scripts import combat_quick_QC


CAMCAN = "./DONNES/CamCAN.md.raw.csv.gz"

models_directory = "ADNI_MODELS"
raw_directory = 'ADNI_RAW'
clean_directory = 'ADNI_CLEAN'
best_directory = 'ADNI_BEST'
metric = "md"
method= "vanilla"

hi


In [2]:
def get_info(mov_data_file):
    [df,bundles] = combat_info.info(mov_data_file)
    nb_hc = int(re.findall('HC\(n=(\d+)',df["DetailInfos"]["Disease"])[0])
    nb_total = df["DetailInfos"]["Number of Subject"]
    nb_sick = nb_total - nb_hc
    return [nb_total,nb_hc,nb_sick]

In [3]:
def get_bundles(mov_data_file):
    return combat_info.get_bundles(mov_data_file)

In [4]:
def fit(mov_data_file):
    mov_data = pd.read_csv(mov_data_file)
    ###########
    ### fit ###
    ###########
    output_model_filename = (
            mov_data.site.unique()[0]
            + "."
            + metric
            + "."
            + method
            + ".model.csv"
        )
    cmd = (
        "scripts/combat_quick_fit.py"
        + " "
        + CAMCAN
        + " "
        + mov_data_file
        + " --out_dir "
        + models_directory
        + " --output_model_filename "
        + output_model_filename
        + " --method "
        + method
    )
    subprocess.call(cmd, shell=True)
    return

In [5]:
def apply(mov_data_file):
    mov_data = pd.read_csv(mov_data_file)
    model = (
            models_directory
            +
            '/'
            +
            mov_data.site.unique()[0]
            + "."
            + metric
            + "."
            + method
            + ".model.csv"
        )
    combat_quick_apply.apply(mov_data_file, model, clean_directory)

In [6]:
def QC(mov_data_file):
    mov_data = pd.read_csv(mov_data_file)

    model_file = (
            models_directory
            +
            '/'
            +
            mov_data.site.unique()[0]
            + "."
            + metric
            + "."
            + method
            + ".model.csv"
    )
    clean_mov_data = (
        clean_directory
        +
        '/'
        +
        mov_data.site.unique()[0]
        + "."
        + metric
        + "."
        + method
        + ".csv.gz"
    )
    return combat_quick_QC.QC(CAMCAN,clean_mov_data, model_file)

In [7]:
def get_csv_res(mov_data_file):
    mov_data = pd.read_csv(mov_data_file)
    model = (
            models_directory
            +
            '/'
            +
            mov_data.site.unique()[0]
            + "."
            + metric
            + "."
            + method
            + ".model.csv"
        )
    return combat_quick_apply.make_best(mov_data_file, model)

In [8]:
"""mat =[]
fields = ["Name","NB Suject","NB HC","NB Sick","Mean Bhattacharrya"]

for filename in sorted(os.listdir(raw_directory)):
    f = os.path.join(raw_directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        if len(fields)==5:
            fields = fields + get_bundles(f)

        print(f)
        infos = [filename]  
        infos = infos + get_info(f)
        #fit(f)
        #apply(f)
        dist = QC(f)
        infos.append(np.mean(dist))
        infos = infos + dist
        mat.append(infos)
        print("---------------------------")

with open('GFG', 'w') as res:
     
    # using csv.writer method from CSV package
    write = csv.writer(res)
     
    write.writerow(fields)
    write.writerows(mat)"""
        


'mat =[]\nfields = ["Name","NB Suject","NB HC","NB Sick","Mean Bhattacharrya"]\n\nfor filename in sorted(os.listdir(raw_directory)):\n    f = os.path.join(raw_directory, filename)\n    # checking if it is a file\n    if os.path.isfile(f):\n        if len(fields)==5:\n            fields = fields + get_bundles(f)\n\n        print(f)\n        infos = [filename]  \n        infos = infos + get_info(f)\n        #fit(f)\n        #apply(f)\n        dist = QC(f)\n        infos.append(np.mean(dist))\n        infos = infos + dist\n        mat.append(infos)\n        print("---------------------------")\n\nwith open(\'GFG\', \'w\') as res:\n     \n    # using csv.writer method from CSV package\n    write = csv.writer(res)\n     \n    write.writerow(fields)\n    write.writerows(mat)'

In [9]:
header = (
        "sid,bundle,metric,mean,site,age,sex,handedness,disease"
    )
combination = np.empty((9, 0))

for filename in sorted(os.listdir(best_directory)):
    f = os.path.join(best_directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        site = get_csv_res(f)
        combination = np.hstack([combination, site])
        print("---------------------------")

np.savetxt("DONNES/adni_compilation.csv.gz", combination.T, delimiter=",", header=header, fmt="%s", comments="")


ADNI_BEST/adni_127_GE_3T.md.raw.csv.gz
---------------------------
ADNI_BEST/adni_129_GE_3T.md.raw.csv.gz
---------------------------
ADNI_BEST/adni_168_Siemens_3T.md.raw.csv.gz
---------------------------
ADNI_BEST/adni_16_GE_3T.md.raw.csv.gz
---------------------------
ADNI_BEST/adni_21_GE_3T.md.raw.csv.gz
---------------------------
ADNI_BEST/adni_27_GE_3T.md.raw.csv.gz
---------------------------
ADNI_BEST/adni_29_GE_3T.md.raw.csv.gz
---------------------------
ADNI_BEST/adni_3_GE_3T.md.raw.csv.gz
---------------------------
ADNI_BEST/adni_41_Siemens_3T.md.raw.csv.gz
---------------------------
ADNI_BEST/adni_7_GE_3T.md.raw.csv.gz
---------------------------
ADNI_BEST/adni_941_Siemens_3T.md.raw.csv.gz
---------------------------
ADNI_BEST/adni_98_GE_3T.md.raw.csv.gz
---------------------------


In [10]:
"""for filename in sorted(os.listdir(best_directory)):
    f = os.path.join(best_directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        dir = "ADNI_BEST_VIZUALIZE/" + filename.split('.')[0]
        cmd = (
            "combat_visualize_data.py"
            + " "
            + f
            + " "
            + CAMCAN
            + " --out_dir "
            + dir
            + " --bundles all"
        )
        subprocess.call(cmd, shell=True)
        print("---------------------------")"""

'for filename in sorted(os.listdir(best_directory)):\n    f = os.path.join(best_directory, filename)\n    # checking if it is a file\n    if os.path.isfile(f):\n        print(f)\n        dir = "ADNI_BEST_VIZUALIZE/" + filename.split(\'.\')[0]\n        cmd = (\n            "combat_visualize_data.py"\n            + " "\n            + f\n            + " "\n            + CAMCAN\n            + " --out_dir "\n            + dir\n            + " --bundles all"\n        )\n        subprocess.call(cmd, shell=True)\n        print("---------------------------")'